In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
column_names = [
    'area', 'perimeter', 'compactness', 'length_of_kernel',
    'width_of_kernel', 'asymmetry_coefficient', 'length_of_kernel_groove',
    'class'
]

In [3]:
data = pd.read_csv('C:/Users/ghom9/Downloads/seeds_dataset.txt', delim_whitespace=True, names=column_names)  # CSV 파일 경로

C:\Users\ghom9\AppData\Local\Temp\ipykernel_19468\1704746712.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv('C:/Users/ghom9/Downloads/seeds_dataset.txt', delim_whitespace=True, names=column_names)  # CSV 파일 경로


In [4]:
# 입력 (X), 출력 (y) 분리
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values - 1  # 클래스가 1, 2, 3으로 되어 있어서 0부터 시작하도록 조정

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# 원-핫 인코딩
y_categorical = to_categorical(y)

In [7]:
# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

In [8]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense

class SeedsClassifier(Model):
    def __init__(self):
        super(SeedsClassifier, self).__init__()
        self.dense1 = Dense(16, activation='relu')
        self.dense2 = Dense(12, activation='relu')
        self.output_layer = Dense(3, activation='softmax')  # 클래스 3개

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)


In [10]:
model = SeedsClassifier()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2)

Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.2054 - loss: 1.0428 - val_accuracy: 0.1471 - val_loss: 1.0230
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3251 - loss: 0.9592 - val_accuracy: 0.3824 - val_loss: 0.9345
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5626 - loss: 0.8824 - val_accuracy: 0.5588 - val_loss: 0.8735
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6846 - loss: 0.8354 - val_accuracy: 0.7647 - val_loss: 0.8180
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7560 - loss: 0.7608 - val_accuracy: 0.7941 - val_loss: 0.7679
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7970 - loss: 0.7382 - val_accuracy: 0.8529 - val_loss: 0.7168
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8609 - loss: 0.7046 - val_accuracy: 0.9118 - val_loss: 0.6743
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8673 - loss: 0.6631 - val_accuracy: 0.9118 - v

In [11]:
loss, acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {acc:.4f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.9211 - loss: 0.1865
Test Accuracy: 0.9286


In [12]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# 예측: 확률 → 클래스 인덱스
y_pred_probs = model.predict(X_test)
y_pred_class = np.argmax(y_pred_probs, axis=1)

# 실제: 원-핫 → 클래스 인덱스
y_test_class = np.argmax(y_test, axis=1)

# 평가 결과 출력
print("Classification Report:")
print(classification_report(y_test_class, y_pred_class, digits=4))

print("Confusion Matrix:")
print(confusion_matrix(y_test_class, y_pred_class))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Classification Report:
              precision    recall  f1-score   support

           0     0.8333    0.9091    0.8696        11
           1     1.0000    1.0000    1.0000        14
           2     0.9375    0.8824    0.9091        17

    accuracy                         0.9286        42
   macro avg     0.9236    0.9305    0.9262        42
weighted avg     0.9311    0.9286    0.9290        42

Confusion Matrix:
[[10  0  1]
 [ 0 14  0]
 [ 2  0 15]]
